In [ ]:

[root@bosprod01 Downloads]# ifconfig -a

inet 192.168.153.139  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod02 Downloads]# ifconfig -a

inet 192.168.153.140  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod03 ~]# ifconfig -a

inet 192.168.153.141  netmask 255.255.255.0  broadcast 192.168.153.255


Add host ips in /etc/hosts file and stop firewalld in all servers 
---------------------------------------------------------------------------

[root@bosprod02 ~]# cat /etc/hosts
192.168.153.139 bosprod01
192.168.153.140 bosprod02
192.168.153.141 bosprod03
[root@bosprod02 ~]# 

systemctl stop firewalld

In [ ]:
Configuring an instance for group replication
==============================================================


[root@bosprod01 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=1
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod01:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod01 ~]# 


[root@bosprod01 ~]# mysqld --validate-config
[root@bosprod01 ~]# 
[root@bosprod01 ~]# systemctl restart mysqld
[root@bosprod01 ~]# 


In [ ]:
User Credentials For Distributed Recovery
============================================================

mysql> set sql_log_bin = 0;
Query OK, 0 rows affected (0.00 sec)

mysql> create user replicator identified by 'Repl@2024';
Query OK, 0 rows affected (0.19 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant connection_admin on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant backup_admin on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> grant group_replication_stream on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> flush privileges;
Query OK, 0 rows affected (0.01 sec)

mysql> set sql_log_bin = 1;
Query OK, 0 rows affected (0.00 sec)

In [ ]:
Launching Group Replication
=============================================


mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';
Query OK, 0 rows affected, 2 warnings (0.03 sec)


mysql> show plugins;
+---------------------------------+----------+--------------------+----------------------+---------+
| Name                            | Status   | Type               | Library              | License |
+---------------------------------+----------+--------------------+----------------------+---------+
| group_replication               | ACTIVE   | GROUP REPLICATION  | group_replication.so | GPL     |
+---------------------------------+----------+--------------------+----------------------+---------+
46 rows in set (0.04 sec)

In [ ]:
Bootstrapping the Group
==============================================

The process of starting a group for the first time is called bootstrapping.
The bootstrap should only be done by a single server, the one that starts the group and only once.

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
Query OK, 0 rows affected (0.01 sec)

mysql> START GROUP_REPLICATION;
ERROR 3092 (HY000): The server is not configured properly to be an active member of the group. 
Please see more details on error log.
mysql> 

2024-06-12T19:29:22.470271Z 0 [ERROR] [MY-011735] [Repl] Plugin group_replication reported: '[GCS] 
The group communication engine failed to test connectivity t
o the local group communication engine on bosprod01:33061. This may be due to one or more invalid 
configuration settings. Double-check your group replication l
ocal address, firewall, SE Linux and TLS configurations and try restarting Group 
Replication on this server.'


Changing SELinux Mode to Permissive
-----------------------------------------------

[root@bosprod01 ~]# sestatus
SELinux status:                 enabled
SELinuxfs mount:                /sys/fs/selinux
SELinux root directory:         /etc/selinux
Loaded policy name:             targeted
Current mode:                   enforcing
Mode from config file:          enforcing
Policy MLS status:              enabled
Policy deny_unknown status:     allowed
Memory protection checking:     actual (secure)
Max kernel policy version:      33
[root@bosprod01 ~]# 


[root@bosprod01 ~]# vi /etc/selinux/config
[root@bosprod01 ~]# 

SELINUX=permissive

reboot 

[root@bosprod01 ~]# sestatus
SELinux status:                 enabled
SELinuxfs mount:                /sys/fs/selinux
SELinux root directory:         /etc/selinux
Loaded policy name:             targeted
Current mode:                   permissive
Mode from config file:          permissive
Policy MLS status:              enabled
Policy deny_unknown status:     allowed
Memory protection checking:     actual (secure)
Max kernel policy version:      33
[root@bosprod01 ~]# 


mysql> SET GLOBAL group_replication_bootstrap_group=ON;
Query OK, 0 rows affected (0.02 sec)

mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (6.15 sec)

mysql> SET GLOBAL group_replication_bootstrap_group=OFF;
Query OK, 0 rows affected (0.00 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
1 row in set (0.11 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
1 row in set (0.00 sec)

mysql> create database bosp;
Query OK, 1 row affected (0.03 sec)

mysql> use bosp;
Database changed
mysql> 
mysql> create table orders ( order_id int primary key , order_details varchar(30) , order_date datetime );
Query OK, 0 rows affected (0.14 sec)

mysql> insert into orders values ( 1 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.33 sec)

mysql> insert into orders values ( 2 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.01 sec)

mysql> insert into orders values ( 3 , 'gjgdjfdffff', now() );
Query OK, 1 row affected (0.00 sec)


In [ ]:
Adding a second instance to the group 
=======================================================

[root@bosprod02 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=2
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod02:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod02 ~]# 


[root@bosprod02 ~]# mysqld --validate-config
[root@bosprod02 ~]# 
[root@bosprod02 ~]# systemctl restart mysqld
[root@bosprod02 ~]# 


mysql> set sql_log_bin = 0;
Query OK, 0 rows affected (0.00 sec)

mysql> create user replicator identified by 'Repl@2024';
Query OK, 0 rows affected (0.19 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> grant connection_admin on *.* to replicator;
Query OK, 0 rows affected (0.02 sec)

mysql> grant backup_admin on *.* to replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> grant group_replication_stream on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

mysql> flush privileges;
Query OK, 0 rows affected (0.00 sec)

mysql> set sql_log_bin = 1;
Query OK, 0 rows affected (0.00 sec)

mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';
Query OK, 0 rows affected, 2 warnings (0.09 sec)

mysql> START GROUP_REPLICATION;
ERROR 3092 (HY000): The server is not configured properly to be an active member of the group. 
Please see more details on error log.
mysql> 

[root@bosprod01 ~]# systemctl stop firewalld
[root@bosprod01 ~]# 

[root@bosprod02 ~]# systemctl stop firewalld
[root@bosprod02 ~]# 

[root@bosprod03 ~]# systemctl stop firewalld
[root@bosprod03 ~]# 


mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (5.91 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
2 rows in set (0.12 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod02  |        3 |
+------------+----------+
1 row in set (0.17 sec)


mysql> create database posdb;
ERROR 1290 (HY000): The MySQL server is running with the --super-read-only option so it cannot execute this statement
mysql> 
mysql> show variables like '%read_only%';
+-----------------------+-------+
| Variable_name         | Value |
+-----------------------+-------+
| innodb_read_only      | OFF   |
| read_only             | ON    |
| super_read_only       | ON    |
| transaction_read_only | OFF   |
+-----------------------+-------+
4 rows in set (0.19 sec)


In [ ]:
Adding third instance to the group 
=======================================================

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
3 rows in set (0.01 sec)


In [ ]:
Restarting a Group 
===========================================

Group Replication is designed to ensure that the database service is continuously available, even if some of the servers 
that form the group are currently unable to participate in it due to planned maintenance or unplanned issues. 
As long as the remaining members are a majority of the group they can elect a new primary and continue to function as a group. 

However, if every member of a replication group leaves the group, and Group Replication is stopped on every member by a 
STOP GROUP_REPLICATION statement or system shutdown, the group now only exists in theory, as a configuration on the members. 
In that situation, to re-create the group, it must be started by bootstrapping as if it was being started for the first time.


suppose we have stoppped group replication in all hosts 
----------------------------------------------------------------------

mysql> STOP GROUP_REPLICATION;   //on all hosts

mysql> select member_host,member_state from performance_schema.replication_group_members;
+-------------+--------------+
| member_host | member_state |
+-------------+--------------+
| bosprod01   | OFFLINE      |
+-------------+--------------+
+-------------+--------------+
| member_host | member_state |
+-------------+--------------+
| bosprod02   | OFFLINE      |
+-------------+--------------+
+-------------+--------------+
| member_host | member_state |
+-------------+--------------+
| bosprod03   | OFFLINE      |
+-------------+--------------+


collect information about the biggest transaction among all servers 
-----------------------------------------------------------------------------------

mysql>  SELECT @@HOSTNAME , @@GLOBAL.GTID_EXECUTED;
+------------+------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                   |
+------------+------------------------------------------+
| bosprod01  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8 |
+------------+------------------------------------------+
+------------+------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                   |
+------------+------------------------------------------+
| bosprod02  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8 |
+------------+------------------------------------------+
+------------+------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                   |
+------------+------------------------------------------+
| bosprod03  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8 |
+------------+------------------------------------------+

mysql> SELECT @@hostname , received_transaction_set FROM performance_schema.replication_connection_status 
WHERE channel_name="group_replication_applier";
+------------+--------------------------------------------+
| @@hostname | received_transaction_set                   |
+------------+--------------------------------------------+
| bosprod01  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1:7-8 |
+------------+--------------------------------------------+
+------------+------------------------------------------+
| @@hostname | received_transaction_set                 |
+------------+------------------------------------------+
| bosprod02  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:7-8 |
+------------+------------------------------------------+
+------------+----------------------------------------+
| @@hostname | received_transaction_set               |
+------------+----------------------------------------+
| bosprod03  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:8 |
+------------+----------------------------------------+


Do some transactions in bosprod03 host and check the information again 
--------------------------------------------------------------------------------------

mysql> insert into orders values (4,'gdjfgff',now()), (5,'ytfffjfhkj',now()) , (6,'etreejh',now());
Query OK, 3 rows affected (0.19 sec)
Records: 3  Duplicates: 0  Warnings: 0

mysql> SELECT @@HOSTNAME , @@GLOBAL.GTID_EXECUTED;
+------------+----------------------------------------------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                                                           |
+------------+----------------------------------------------------------------------------------+
| bosprod03  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8,
e89f4b84-28e9-11ef-bbb7-000c29009988:1 |
+------------+----------------------------------------------------------------------------------+
+------------+------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                   |
+------------+------------------------------------------+
| bosprod01  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8 |
+------------+------------------------------------------+
+------------+------------------------------------------+
| @@HOSTNAME | @@GLOBAL.GTID_EXECUTED                   |
+------------+------------------------------------------+
| bosprod02  | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-8 |
+------------+------------------------------------------+


Use the member that has the biggest transaction set to bootstrap the group ( bosprod03 )
---------------------------------------------------------------------------------------------

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
Query OK, 0 rows affected (0.01 sec)

mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (1.69 sec)

mysql> SET GLOBAL group_replication_bootstrap_group=OFF;
Query OK, 0 rows affected (0.00 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
1 row in set (0.00 sec)


Add each of the other members back into the group, in any order, by issuing a START GROUP_REPLICATION 
----------------------------------------------------------------------------------------------------------

mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (5.03 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | RECOVERING   | SECONDARY   | 8.0.30         | XCom                       |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
2 rows in set (0.10 sec)


mysql> START GROUP_REPLICATION;
Query OK, 0 rows affected (5.56 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION | MEMBER_COMMUNICATION_STACK |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | RECOVERING   | SECONDARY   | 8.0.30         | XCom                       |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | RECOVERING   | SECONDARY   | 8.0.30         | XCom                       |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     | 8.0.30         | XCom                       |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+----------------+----------------------------+
3 rows in set (0.04 sec)


mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
--------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE 
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
--------------+----------------------------+
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+
------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+
------------+----------------------------+
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ERROR        |
+---------------------------+--------------------------------------+-------------+-------------+--------------+

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------
---------------+----------------------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE 
+---------------------------+--------------------------------------+-------------+-------------+--------------
---------------+----------------------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ERROR          |
+---------------------------+--------------------------------------+-------------+-------------+--------------


stop and start the group_replication in replicas 
------------------------------------------------------------

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | SECONDARY   |                  |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     |

mysql> select @@hostname,count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod03  |        6 |
+------------+----------+
+------------+----------+
| bosprod02  |        6 |
+------------+----------+
+------------+----------+
| bosprod01  |        6 |
+------------+----------+


mysql> select * from performance_schema.replication_applier_status;
+----------------------------+---------------+-----------------+----------------------------+
| CHANNEL_NAME               | SERVICE_STATE | REMAINING_DELAY | COUNT_TRANSACTIONS_RETRIES |
+----------------------------+---------------+-----------------+----------------------------+
| group_replication_applier  | ON            |            NULL |                          0 |
| group_replication_recovery | OFF           |            NULL |                          0 |
+----------------------------+---------------+-----------------+----------------------------+
2 rows in set (0.03 sec)

mysql> select CHANNEL_NAME,MEMBER_ID,TRANSACTIONS_COMMITTED_ALL_MEMBERS from 
performance_schema.replication_group_member_stats;

+---------------------------+--------------------------------------+------------------------------------------
| CHANNEL_NAME              | MEMBER_ID                            | TRANSACTIONS_COMMITTED_ALL_MEMBERS         
+---------------------------+--------------------------------------+--------------------------------------------
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-13,
e89f4b84-28e9-11ef-bbb7-000c29009988:1 |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-13,
e89f4b84-28e9-11ef-bbb7-000c29009988:1 |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | 5842bb1d-28ef-11ef-9a91-000c293ea0bc:1-13,
e89f4b84-28e9-11ef-bbb7-000c29009988:1 |
+---------------------------+--------------------------------------+--------------------------------------------

In [ ]:
Changing primary in group replication
======================================================

Primary member can be changed using the group_replication_set_as_primary() function, 
which can be can be run on any member of the group.

Any uncommitted transactions that the group is waiting on must be committed, rolled back, or terminated before the 
operation can complete. 
You can specify a timeout from 0 seconds (immediately) up to 3600 seconds (60 minutes) for transactions that are running
when you use the function. There is no default setting for the timeout, so if you do not set it, there is no upper limit to the wait time, 
and new transactions can start during that time.


run a big transaction in primary ( bosprod03 )
-------------------------------------------------

mysql> call sp_insert_orders(7);


change primary host to bosprod02
-----------------------------------------------

mysql> SELECT group_replication_set_as_primary('e5406206-28e9-11ef-b28f-000c293ea0bc');
+--------------------------------------------------------------------------+
| group_replication_set_as_primary('e5406206-28e9-11ef-b28f-000c293ea0bc') |
+--------------------------------------------------------------------------+
| Primary server switched to: e5406206-28e9-11ef-b28f-000c293ea0bc         |
+--------------------------------------------------------------------------+
1 row in set (0.50 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | SECONDARY   |               |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+


check the transaction status in bosprod03
-----------------------------------------------------

mysql> call sp_insert_orders(7);
Query OK, 0 rows affected (8.20 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod03  |   100006 |
+------------+----------+
1 row in set (0.01 sec)


Do a bihgger transaction and include wait time in primary switching
--------------------------------------------------------------------------------

mysql> call sp_insert_orders(100007);

mysql> SELECT group_replication_set_as_primary('8da6031a-28e9-11ef-a875-000c29051378',180);  // 3 minutes wait time

mysql> call sp_insert_orders(100007);
Query OK, 0 rows affected (1 min 9.31 sec)

mysql> SELECT group_replication_set_as_primary('8da6031a-28e9-11ef-a875-000c29051378',180);
+------------------------------------------------------------------------------+
| group_replication_set_as_primary('8da6031a-28e9-11ef-a875-000c29051378',180) |
+------------------------------------------------------------------------------+
| Primary server switched to: 8da6031a-28e9-11ef-a875-000c29051378             |
+------------------------------------------------------------------------------+
1 row in set (1 min 41.73 sec)

mysql> call sp_insert_orders(1100007);
ERROR 1290 (HY000): The MySQL server is running with the --super-read-only option so it cannot execute this statement
mysql> 

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+

In [ ]:
Changing mode to multi-primary in group replication
=========================================================


mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+


mysql> SELECT group_replication_switch_to_multi_primary_mode();
+--------------------------------------------------+
| group_replication_switch_to_multi_primary_mode() |
+--------------------------------------------------+
| Mode switched to multi-primary successfully.     |
+--------------------------------------------------+

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+

mysql> SELECT group_replication_switch_to_single_primary_mode();
+---------------------------------------------------+
| group_replication_switch_to_single_primary_mode() |
+---------------------------------------------------+
| Mode switched to single-primary successfully.     |
+---------------------------------------------------+
1 row in set (0.96 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE 
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------


In [ ]:
Unexpected primary server down 
================================================


[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 


mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+

mysql> call sp_insert_orders(1100007);

mysql> select @@hostname,count(*) from orders;
| bosprod02  |  2100006 |

mysql> select @@hostname,count(*) from orders;
| bosprod03  |  2100006 |


old primary is back online , add to the group 
--------------------------------------------------

[root@bosprod01 ~]# systemctl start mysqld
[root@bosprod01 ~]# 


mysql> start group_replication;
Query OK, 0 rows affected (8.03 sec)

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | RECOVERING   | SECONDARY   |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+

mysql> select @@hostname,count(*) from orders;
+------------+----------+
| bosprod01  |  2100006 |
+------------+----------+

In [ ]:
Group Replication Offline Upgrade 
=================================================

To perform offline upgrade , remove each member from the group and shutdown all the members, perform an upgrade and restart the group.

In multi-primary mode , group members can be shutdown in any order.

In single-primary mode , secondary should be shutdown and then finally primary.

mysql> select member_host,member_state,member_role,member_version from 
    -> performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| member_host | member_state | member_role | member_version |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | SECONDARY   | 8.0.30         |
| bosprod02   | ONLINE       | PRIMARY     | 8.0.30         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.30         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.09 sec)


mysql> stop group_replication;
[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 

mysql> stop group_replication;
[root@bosprod03 ~]# systemctl stop mysqld
[root@bosprod03 ~]# 

mysql> stop group_replication;
[root@bosprod02 ~]# systemctl stop mysqld
[root@bosprod02 ~]# 


Minor version upgarde -  8.0.30 -> 8.0.32  , in-place

[root@bosprod01 ~]# yum install mysql-community-server-*8.0.32* --skip-broken

Preparing        :                                                   
Running scriptlet: mysql-community-icu-data-files-8.0.32-1.el8.x86_64
Upgrading        : mysql-community-icu-data-files-8.0.32-1.el8.x86_64
Upgrading        : mysql-community-common-8.0.32-1.el8.x86_64        
Running scriptlet: mysql-community-server-8.0.32-1.el8.x86_64        
Upgrading        : mysql-community-server-8.0.32-1.el8.x86_64        
Running scriptlet: mysql-community-server-8.0.32-1.el8.x86_64        
Running scriptlet: mysql-community-server-8.0.30-1.el8.x86_64        

Upgraded:
  mysql-community-common-8.0.32-1.el8.x86_64                mysql-community-icu-data-files-8.0.32-1.el8.x86_64 
  mysql-community-server-8.0.32-1.el8.x86_64               

Complete!
[root@bosprod01 ~]# 

[root@bosprod01 ~]# systemctl start mysqld
[root@bosprod01 ~]# 

mysql> select @@hostname,@@version;
+------------+-----------+
| @@hostname | @@version |
+------------+-----------+
| bosprod01  | 8.0.32    |
+------------+-----------+
+------------+-----------+
| @@hostname | @@version |
+------------+-----------+
| bosprod02  | 8.0.32    |
+------------+-----------+
+------------+-----------+
| @@hostname | @@version |
+------------+-----------+
| bosprod03  | 8.0.32    |
+------------+-----------+


mysql> start group_replication;
ERROR 3092 (HY000): The server is not configured properly to be an active member of the group. 
Please see more details on error log.

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
mysql> START GROUP_REPLICATION;
mysql> SET GLOBAL group_replication_bootstrap_group=OFF;

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION
    -> from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
+-------------+--------------+-------------+----------------+

mysql> start group_replication;
mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION
    -> from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | RECOVERING   | SECONDARY   | 8.0.32         |
+-------------+--------------+-------------+----------------+
2 rows in set (0.22 sec)


mysql> start group_replication;
mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION
    -> from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | RECOVERING   | SECONDARY   | 8.0.32         |
| bosprod03   | RECOVERING   | SECONDARY   | 8.0.32         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.01 sec)


mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION 
       from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | ONLINE       | SECONDARY   | 8.0.32         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.32         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.00 sec)

In [ ]:
Group Replication Online Upgrade 
=================================================


To perform online upgrade , remove a member from the group , shutdown and upgrade it and then rejoin it to the group.
so when upgrading one instance other 2 instances are available.

while upgrading single-primary group it is recommended to first upgrade all the secondaries and then the primary.


mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION 
    ->        from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | ONLINE       | SECONDARY   | 8.0.32         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.32         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.34 sec)


Minor version upgrade -  8.0.32 -> 8.0.34  , in-place
-------------------------------------------------------------------


upgrade the first secondary instance 
--------------------------------------------------

mysql> stop group_replication;
Query OK, 0 rows affected (3.40 sec)

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod03   | OFFLINE      |             |                |
+-------------+--------------+-------------+----------------+

do some transactions to simulate the server group is up while on of its member is being Upgraded
----------------------------------------------------------------------------------------------------

mysql> select @@hostname , count(*) from orders;
| bosprod01  |  2100006 |

mysql> call sp_insert_orders(2100007);

mysql> select @@hostname , count(*) from orders;
| bosprod01  |  3100006 |


stop the secondary member and upgrade 
-----------------------------------------------------

[root@bosprod03 ~]# systemctl stop mysqld
[root@bosprod03 ~]# 
[root@bosprod03 ~]# yum install mysql-community-server-*8.0.34* --skip-broken

Upgraded:
  mysql-community-common-8.0.34-1.el8.x86_64   mysql-community-icu-data-files-8.0.34-1.el8.x86_64                
  mysql-community-server-8.0.34-1.el8.x86_64                

Complete!

[root@bosprod03 ~]# systemctl start mysqld
[root@bosprod03 ~]# 
mysql> select @@hostname,@@version;
+------------+-----------+
| bosprod03  | 8.0.34    |
+------------+-----------+

rejoin the secondary member after upgrade to the group 
------------------------------------------------------------

mysql> start group_replication;
Query OK, 0 rows affected (20.96 sec)

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | ONLINE       | SECONDARY   | 8.0.32         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.34         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.01 sec)

mysql> select @@hostname , count(*) from orders;
| bosprod03  |  3100006 |



upgrade the second secondary instance 
--------------------------------------------------

mysql> stop group_replication;
Query OK, 0 rows affected (7.81 sec)

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod02   | OFFLINE      |             |                |
+-------------+--------------+-------------+----------------+
1 row in set (0.10 sec)


mysql> call sp_insert_orders(3100007);

mysql> select @@hostname , count(*) from orders;
| bosprod01  |  4100006 |


[root@bosprod02 ~]# systemctl stop mysqld
[root@bosprod02 ~]# 
[root@bosprod02 ~]# yum install mysql-community-server-*8.0.34* --skip-broken

Upgraded:
  mysql-community-common-8.0.34-1.el8.x86_64   mysql-community-icu-data-files-8.0.34-1.el8.x86_64      
  mysql-community-server-8.0.34-1.el8.x86_64      

Complete!
[root@bosprod02 ~]# 

[root@bosprod02 ~]# systemctl start mysqld
[root@bosprod02 ~]# 

mysql> select @@hostname , @@version;
| bosprod02  | 8.0.34    |

mysql> start group_replication;

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | PRIMARY     | 8.0.32         |
| bosprod02   | ONLINE       | SECONDARY   | 8.0.34         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.34         |
+-------------+--------------+-------------+----------------+

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| bosprod02  |  4100006 |
+------------+----------+


upgrade the final primary instance 
--------------------------------------------------

mysql> stop group_replication;
Query OK, 0 rows affected (7.68 sec)

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | OFFLINE      |             |                |
+-------------+--------------+-------------+----------------+
1 row in set (0.07 sec)

+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod02   | ONLINE       | PRIMARY     | 8.0.34         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.34         |
+-------------+--------------+-------------+----------------+



[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 
[root@bosprod01 ~]# yum install mysql-community-server-*8.0.34* --skip-broken

Upgraded:
  mysql-community-common-8.0.34-1.el8.x86_64    mysql-community-icu-data-files-8.0.34-1.el8.x86_64   
  mysql-community-server-8.0.34-1.el8.x86_64   

Complete!
[root@bosprod01 ~]# 



[root@bosprod01 ~]# systemctl start mysqld
[root@bosprod01 ~]# 
mysql> select @@hostname,@@version;
+------------+-----------+
| bosprod01  | 8.0.34    |
+------------+-----------+

mysql> start group_replication;

mysql> select MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE,MEMBER_VERSION from performance_schema.replication_group_members;
+-------------+--------------+-------------+----------------+
| MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE | MEMBER_VERSION |
+-------------+--------------+-------------+----------------+
| bosprod01   | ONLINE       | SECONDARY   | 8.0.34         |
| bosprod02   | ONLINE       | PRIMARY     | 8.0.34         |
| bosprod03   | ONLINE       | SECONDARY   | 8.0.34         |
+-------------+--------------+-------------+----------------+
3 rows in set (0.00 sec)